<a href="https://colab.research.google.com/github/xslittlemaggie/Other-ML-DL-Algorithm-notes/blob/master/Copy_of_Data_Preprocessing_%26_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center> Five steps of data mining<center><h1>

# Part One: Loading data

From kaggles, local, sklearn built-in datasets, etc.

# Part Two & Three: Data Preprocessing & Feature Engineering

The purpose of data processing is to clean the data for model building.

package **preprocessing**: include almost all data preprocessing methods

package ** feature_selection**: include a lot of feature selection methods

package **decomposition**: e.g. PCA

In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import pandas as pd

# 1. 无量纲化

大部分情况下使用标准化

The purpoes is to increase model accuracy

### 1.1 Normalization (归一化)

取最大值和最小值， 对异常值非常敏感

In [0]:
data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]
pd.DataFrame(data)

In [0]:
scaler = MinMaxScaler()
result = scaler.fit_transform(data)
result

In [0]:
scaler.inverse_transform(result)

In [0]:
# fit the data into a particular range

scaler = MinMaxScaler(feature_range = [5, 10])
result = scaler.fit_transform(data)
result

### 1.2. Standardization(标准化)

Z-score

In [0]:
scaler = StandardScaler()
result = scaler.fit_transform(data)
result

In [0]:
print("Mean: {}".format(scaler.mean_))
print("Variance: {}".format(scaler.var_))

In [0]:
print("Mean: {}".format(result.mean()))
print("Variance: {}".format(result.std()))

In [0]:
scaler.inverse_transform(result)

# 2. Missing values

load the titanic dataset from kaggle for practice

In [0]:
import os

os.environ['KAGGLE_USERNAME'] = "liulihuang" # username from the json file 
os.environ['KAGGLE_KEY'] = "7adfc6c4e6c5eec087031fbb7397aee5" # key from the json file (This key is incorrect) 

In [0]:
!pip install -q kaggle
#!kaggle datasets list -s titanic  # It will list the 20 datasets including "titanic" from kaggle
!kaggle datasets download -d kittisaks/testtitanic -p /content/
!unzip -q /content/titanic.zip -d /content/titanic/
!unzip -q /content/testtitanic.zip -d /content/titanic/

In [0]:
data = pd.read_csv("/content/titanic/titanic_data.csv")
data.head()

In [0]:
columns = ["Age", "Sex", "Embarked", "Survived"]
data = data[columns]
data.head()

In [0]:
data.shape

### Method 1: impute.SimpleImputer
np.nan = missing values
class sklearn.impute.SimpleImputer(missing_values = nan, strategy = "mean", fill_value = None, copy = True)


- missing_values: nan, or other values I want to replace
- strategy: "mean", "median", "most_frequent", "constant"
- fill_value: when the strategy = "constant"
- copy: default = True

In [0]:
data.info()

The feature "Age", "Survived" have missing values

#### 1. Replacing missing values from Age with median

In [0]:
Age = data.loc[:, "Age"].values.reshape(-1, 1)  # reshape to transform 1 dim to 2 dim
Age[:5]

In [0]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer()  # mean by default
imp_median = SimpleImputer(strategy = "median")
imp_0 = SimpleImputer(strategy = "constant", fill_value = 0)


In [0]:
imp_mean = imp_mean.fit_transform(Age)
imp_median = imp_median.fit_transform(Age)
imp_0 = imp_0.fit_transform(Age)

In [0]:
imp_mean[:5]

In [0]:
imp_median[:5]

In [0]:
imp_0[:5]

In [0]:
# replace the data with medain
data.loc[:, "Age"] = imp_median

In [0]:
data.info()

#### 2. Repalcing variable Embarked with mode

In [0]:
Embarked = data.loc[:, "Embarked"].values.reshape(-1, 1)
imp_mode = SimpleImputer(strategy = "most_frequent")
data.loc[:, "Embarked"] = imp_mode.fit_transform(Embarked)
data.info()

### Method 2: replace missing values with pandas, numpy

In [0]:
data = pd.read_csv("/content/titanic/titanic_data.csv")
columns = ["Age", "Sex", "Embarked", "Survived"]
data = data[columns]
data.head()

In [0]:
data.info()

In [0]:
# replace missing values with fillna 
data.loc[:, "Age"] = data.loc[:, "Age"].fillna(data.loc[:, "Age"].median())

In [0]:
Embarked = data.loc[:, "Embarked"].values.reshape(-1, 1)
imp_mode = SimpleImputer(strategy = "most_frequent")
data.loc[:, "Embarked"] = imp_mode.fit_transform(Embarked)

In [0]:
# data.dropna(axis = 0, inplace = True)
# remove all rows with missing values, (axis = 1) --> remove all columns with missing values

In [0]:
data.info()

# 3. categorical features:编码和哑变量

### 3.1 Label encoder

In [0]:
from sklearn.preprocessing import LabelEncoder
data.iloc[:, -1] = LabelEncoder().fit_transform(data.iloc[:, -1])

In [0]:
data.head()

### 3.2 Feature encoder (only for practice, not appropriate for this data)

In [0]:
from sklearn.preprocessing import OrdinalEncoder
data_ = data.copy()

In [0]:
data_.head()

In [0]:
OrdinalEncoder().fit(data_.iloc[:, 1:-1]).categories_

In [0]:
data_.iloc[:, 1:-1] = OrdinalEncoder().fit_transform(data_.iloc[:, 1:-1])
data_.head()

### 3.3 OneHotEncoder to deal with nominal categorical data

In [0]:
from sklearn.preprocessing import OneHotEncoder
X = data.iloc[:, 1:-1]
X.head()

In [0]:
result = OneHotEncoder(categories = "auto").fit_transform(X).toarray()
#.get_feature_names()

In [0]:
newdata = pd.concat([data, pd.DataFrame(result)], axis = 1)
newdata.head()

In [0]:
newdata.drop(["Sex", "Embarked"], axis = 1, inplace = True)
newdata.columns = ["Age", "Survived", "Female", "Male", "Embarked_C", "Embarked_Q", "Embarked_S"]
newdata.head()

## 4 Continuous feature, e.g. bin

In [0]:
data_2 = data.copy()

In [0]:
from sklearn.preprocessing import Binarizer #特征专用,不让 1 dim 

X = data_2.iloc[:, 0].values.reshape(-1, 1)

In [0]:
transformer = Binarizer(threshold = 30).fit_transform(X)
transformer[:10]

## 5. 连续型变量进行分箱

### preprocessing.KBinsDiscretizer

- n_bins, 默认分箱为5， 一次会被运用到所有倒入的特征
- encoder， 默认“onehot”
- strategy, 默认“quantile"， 等位分箱， 每个特征中的每个箱内的数量样本都相同
  - "uniform": 等宽分箱， 即每个特征中的每个箱的最大值之间的差为（max-min）/n_bins
  - "kmeans":按聚类分箱， 每个箱内中的值到最近的一维k均值聚类的簇心的距离都相同

In [0]:
from sklearn.preprocessing import KBinsDiscretizer

X = data.iloc[:, 0].values.reshape(-1, 1)

In [0]:
est = KBinsDiscretizer(n_bins = 3, encode = "ordinal", strategy = "uniform")
est.fit_transform(X).ravel()[:20]

In [0]:
set(est.fit_transform(X).ravel())

In [0]:
est = KBinsDiscretizer(n_bins = 3, encode = "onehot", strategy = "uniform")
est.fit_transform(X).toarray()

## Part Four: Model building (ignore)


## Part Five: Model evaluation & application (ignore)

### 分箱方法
URL_0 = "http://www.voidcn.com/article/p-hpqunnqv-bro.html" 

URL_1 = "https://zhuanlan.zhihu.com/p/52312186"

URL_2 = "https://blog.csdn.net/Pylady/article/details/78882220"

URL_3 = "https://www.jianshu.com/p/0805f185ecdf"

URL_4 = "https://zhuanlan.zhihu.com/p/35284849"